In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
data=pd.read_csv('/kaggle/input/rotten-tomatoes-reviews/rt_reviews.csv',encoding='latin1')
data.head()

In [ ]:
data['Freshness']=data['Freshness'].replace(['fresh','rotten'],[1,0])

In [ ]:
data['Freshness']

In [ ]:
data.shape

In [ ]:
data.info()

In [ ]:
X=data['Review']
y=data['Freshness']

In [ ]:
from keras.preprocessing.text import Tokenizer

In [ ]:
tokenizer=Tokenizer()
#Tokenizer(oov_token='<nothing>') oove_token mean if a word does not present in training data and this word apear in test data,
# then the word will be replaced with this value
tokenizer.fit_on_texts(X)
tokenizer.word_index

In [ ]:
tokenizer.word_counts

In [ ]:
sequences=tokenizer.texts_to_sequences(X)
sequences[0:7]

In [ ]:
from keras.utils import pad_sequences
sequences=pad_sequences(sequences,padding='post')


In [ ]:
from keras import Sequential
from keras.layers import Dense,Flatten,Embedding,SimpleRNN

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(sequences,y,random_state=5)

In [ ]:
sequences

In [ ]:
len(sequences[0])

In [ ]:
model=Sequential()
model.add(Embedding(5000000,2))
model.add(SimpleRNN(32,input_shape=(55,1),return_sequences=False))
model.add(Dense(16))
model.add(Dense(1,activation='sigmoid'))
model.summary()

In [ ]:
callback = EarlyStopping(
    monitor="val_accuracy",
    min_delta=0.0001,
    patience=3,
    verbose=1,
    mode="auto",
    baseline=None,
    restore_best_weights=True
)

In [ ]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

model.fit(X_train,y_train,epochs=10,validation_data=(X_test,y_test),batch_size=600,callbacks=callback)

In [ ]:
import numpy as np
# input_data=np.array(X_test[100])

y_predict=model.predict(X_test)

In [ ]:
def replace_values(value):
  if value<0.50:
    return 0
  else:
    return 1


In [ ]:
y_predict=pd.DataFrame(y_predict)
y_predict.columns=['ypredict']
y_predict['ypredict']=y_predict['ypredict'].apply(replace_values)
y_predict['ypredict'][0:10]

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

In [ ]:
sns.heatmap(confusion_matrix(y_test,y_predict),annot=True)
